# 初始化目录

In [1]:
# 加入plot支持
%matplotlib inline

# 把当前路径的上级路径作为root
import os
import sys
cur_dir = os.getcwd()
pkg_rootdir = os.path.dirname(cur_dir)
sys.path.append(pkg_rootdir)
os.chdir(pkg_rootdir)
print("当前工作路径：",pkg_rootdir)

当前工作路径： /Users/piginzoo/workspaces/mlstock


In [3]:
from mlstock.data import data_filter
df = data_filter.filter_stocks()
print(df)

        ts_code  name area industry market list_status  list_date delist_date  \
0     000001.SZ  平安银行   深圳       银行     主板           L 1991-04-03        None   
1     000002.SZ   万科A   深圳     全国地产     主板           L 1991-01-29        None   
4     000006.SZ  深振业A   深圳     区域地产     主板           L 1992-04-27        None   
5     000007.SZ   全新好   深圳     酒店餐饮     主板           L 1992-04-13        None   
6     000008.SZ  神州高铁   北京     运输设备     主板           L 1992-05-07        None   
...         ...   ...  ...      ...    ...         ...        ...         ...   
4283  605398.SH  新炬网络   上海     软件服务     主板           L 2021-01-21        None   
4284  605399.SH  晨光新材   江西     化工原料     主板           L 2020-08-04        None   
4285  605488.SH  福莱新材   浙江       塑料     主板           L 2021-05-13        None   
4286  605499.SH  东鹏饮料   深圳      软饮料     主板           L 2021-05-27        None   
4287  605500.SH  森林包装   浙江       造纸     主板           L 2020-12-22        None   

         period  
0    1142

# 过滤股票集合

In [5]:
from mlstock.ml.main import main
df = main("20180101","20220101",20)

In [6]:
df

,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
0,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180105,...,1408.5942,5.3160,0.000000,8108913.30,1.092060e+07,NaN,NaN,0.0268,-0.026800,-0.002003
1,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180112,...,1424.5406,15.9450,0.018797,8704112.30,1.152740e+07,NaN,NaN,0.0208,-0.002003,0.077951
2,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180119,...,1564.8685,8.5100,0.092251,12942411.10,1.865851e+07,NaN,NaN,0.0143,0.077951,-0.073076
3,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180126,...,1509.5878,-15.9450,-0.050676,11456924.00,1.660992e+07,NaN,NaN,0.0224,-0.073076,0.025100
4,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180202,...,1491.5153,2.1270,0.000000,8115143.20,1.129147e+07,NaN,NaN,-0.0251,0.025100,-0.067172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211203,...,135.0226,0.2854,-0.004799,80642.89,2.682903e+05,-7.945102,25.185671,0.0084,-0.013199,-0.014221
4073,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211210,...,137.1839,0.4486,0.017179,108560.80,3.622003e+05,-7.486604,32.342832,0.0314,-0.014221,0.052493
4074,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211217,...,142.0775,0.0408,0.032593,202989.50,6.984993e+05,-6.521659,75.344034,-0.0199,0.052493,0.006413
4075,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211224,...,140.8133,1.2642,-0.000287,105031.92,3.652011e+05,-5.711379,110.195883,-0.0067,0.006413,0.057524


# 划分训练测试集

In [7]:
trade_dates = df.trade_date.sort_values().unique()

In [18]:
import math
div_num = math.ceil(len(trade_dates)*0.8)
train_dates = trade_dates[:div_num]
test_dates =  trade_dates[div_num:]
df_train = df[df.trade_date.apply(lambda x: x in train_dates)]
df_test = df[df.trade_date.apply(lambda x: x in test_dates)]

df_test.pct_chg_hs300.max()
# df_test.shape

0.0364

In [19]:
df_train

,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
0,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180105,...,1408.5942,5.3160,0.000000,8108913.30,1.092060e+07,NaN,NaN,0.0268,-0.026800,-0.002003
1,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180112,...,1424.5406,15.9450,0.018797,8704112.30,1.152740e+07,NaN,NaN,0.0208,-0.002003,0.077951
2,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180119,...,1564.8685,8.5100,0.092251,12942411.10,1.865851e+07,NaN,NaN,0.0143,0.077951,-0.073076
3,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180126,...,1509.5878,-15.9450,-0.050676,11456924.00,1.660992e+07,NaN,NaN,0.0224,-0.073076,0.025100
4,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180202,...,1491.5153,2.1270,0.000000,8115143.20,1.129147e+07,NaN,NaN,-0.0251,0.025100,-0.067172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210219,...,154.6888,5.4700,0.035631,75696.56,2.970532e+05,-12.560730,28.275469,-0.0050,0.040631,0.045585
4032,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210226,...,156.2461,-1.0380,-0.030915,171648.18,6.774649e+05,-13.165505,36.085603,-0.0765,0.045585,0.056864
4033,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210305,...,160.1992,1.6770,0.042964,134960.12,5.399992e+05,-12.691759,46.793709,-0.0139,0.056864,-0.002073
4034,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210312,...,157.6836,0.2795,-0.024173,111698.55,4.448124e+05,-12.639293,47.389139,-0.0221,-0.002073,0.060972


# 去除上市12周内的数据（太剧烈波动）

In [20]:
import pandas as pd
a = pd.to_datetime(df_train.trade_date, format='%Y%m%d')
b = pd.to_datetime(df_train.list_date, format='%Y%m%d')
df_train = df_train[a-b > pd.Timedelta(12,unit='w')] 
df_train

,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
0,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180105,...,1408.5942,5.3160,0.000000,8108913.30,1.092060e+07,NaN,NaN,0.0268,-0.026800,-0.002003
1,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180112,...,1424.5406,15.9450,0.018797,8704112.30,1.152740e+07,NaN,NaN,0.0208,-0.002003,0.077951
2,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180119,...,1564.8685,8.5100,0.092251,12942411.10,1.865851e+07,NaN,NaN,0.0143,0.077951,-0.073076
3,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180126,...,1509.5878,-15.9450,-0.050676,11456924.00,1.660992e+07,NaN,NaN,0.0224,-0.073076,0.025100
4,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180202,...,1491.5153,2.1270,0.000000,8115143.20,1.129147e+07,NaN,NaN,-0.0251,0.025100,-0.067172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210219,...,154.6888,5.4700,0.035631,75696.56,2.970532e+05,-12.560730,28.275469,-0.0050,0.040631,0.045585
4032,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210226,...,156.2461,-1.0380,-0.030915,171648.18,6.774649e+05,-13.165505,36.085603,-0.0765,0.045585,0.056864
4033,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210305,...,160.1992,1.6770,0.042964,134960.12,5.399992e+05,-12.691759,46.793709,-0.0139,0.056864,-0.002073
4034,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210312,...,157.6836,0.2795,-0.024173,111698.55,4.448124e+05,-12.639293,47.389139,-0.0221,-0.002073,0.060972


In [21]:
a = pd.to_datetime(df_test.trade_date, format='%Y%m%d')
b = pd.to_datetime(df_test.list_date, format='%Y%m%d')
df_test = df_test[a-b > pd.Timedelta(12,unit='w')] 
df_test

,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
165,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20210326,...,2304.2460,43.3000,0.032731,4298952.37,9.043860e+06,212.158945,-12.549746,0.0062,0.026531,-0.007471
166,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20210402,...,2418.6255,-31.1000,0.017029,3426882.06,7.418832e+06,198.128302,3.501752,0.0245,-0.007471,0.015198
167,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20210409,...,2394.1948,-28.8800,-0.009302,1700697.18,3.665203e+06,182.460000,28.103318,-0.0245,0.015198,-0.035126
168,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20210416,...,2260.9373,-11.1000,-0.048826,3097949.11,6.350587e+06,154.780038,19.138817,-0.0137,-0.035126,0.115456
169,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20210423,...,2551.8830,34.4400,0.149556,5197078.95,1.157247e+07,171.247562,65.796306,0.0341,0.115456,0.002300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4072,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211203,...,135.0226,0.2854,-0.004799,80642.89,2.682903e+05,-7.945102,25.185671,0.0084,-0.013199,-0.014221
4073,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211210,...,137.1839,0.4486,0.017179,108560.80,3.622003e+05,-7.486604,32.342832,0.0314,-0.014221,0.052493
4074,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211217,...,142.0775,0.0408,0.032593,202989.50,6.984993e+05,-6.521659,75.344034,-0.0199,0.052493,0.006413
4075,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20211224,...,140.8133,1.2642,-0.000287,105031.92,3.652011e+05,-5.711379,110.195883,-0.0067,0.006413,0.057524


In [30]:
print(df_train.target.count(),df_train.shape[0],df_train.target.count()/df_train.shape[0])
df_train = df_train[~df_train.target.isnull()]
df_test = df_test[~df_test.target.isnull()]
print(df_train.target.count(),df_train.shape[0],df_train.target.count()/df_train.shape[0])

3253 3253 1.0
3253 3253 1.0


# 去极值

In [41]:
# 保留feature
feature_names = ['MACD','KDJ']
df_features = df_train[feature_names]
# 每列都求中位数，和中位数之差的绝对值的中位数
df_median = df_features.median()
df_scope = df_features.apply(lambda x: x - df_median[x.name]).abs().median()
df_scope
def scaller(x):
    _max = df_median[x.name] + 5 * df_scope[x.name]
    _min = df_median[x.name] - 5 * df_scope[x.name]
    x = x.apply(lambda v: _min if v < _min else v)
    x = x.apply(lambda v: _max if v > _max else v)
    return x
df_features = df_features.apply(scaller)
df_train[feature_names] = df_features
df_train

,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
0,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180105,...,1408.5942,5.3160,0.000000,8108913.30,1.092060e+07,NaN,NaN,0.0268,-0.026800,-0.002003
1,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180112,...,1424.5406,15.9450,0.018797,8704112.30,1.152740e+07,NaN,NaN,0.0208,-0.002003,0.077951
2,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180119,...,1564.8685,8.5100,0.092251,12942411.10,1.865851e+07,NaN,NaN,0.0143,0.077951,-0.073076
3,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180126,...,1509.5878,-15.9450,-0.050676,11456924.00,1.660992e+07,NaN,NaN,0.0224,-0.073076,0.025100
4,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180202,...,1491.5153,2.1270,0.000000,8115143.20,1.129147e+07,NaN,NaN,-0.0251,0.025100,-0.067172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210219,...,154.6888,5.4700,0.035631,75696.56,2.970532e+05,-12.560730,28.275469,-0.0050,0.040631,0.045585
4032,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210226,...,156.2461,-1.0380,-0.030915,171648.18,6.774649e+05,-13.165505,36.085603,-0.0765,0.045585,0.056864
4033,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210305,...,160.1992,1.6770,0.042964,134960.12,5.399992e+05,-12.691759,46.793709,-0.0139,0.056864,-0.002073
4034,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210312,...,157.6836,0.2795,-0.024173,111698.55,4.448124e+05,-12.639293,47.389139,-0.0221,-0.002073,0.060972


# 空值按照行业平均填充 TODO

In [48]:
df_features_mean = df_train.groupby(by='industry').mean()
df_features_mean
# df_mean_industry = df_mean_industry.fillna(df_mean_industry.mean())
# df_mean_industry.columns = [x+'_mean' for x in df_mean_industry.columns]
# df_mean_industry = df_mean_industry.reset_index()

,open,high,low,close,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
industry,,,,,,,,,,,,,,
IT设备,182.396533,193.183847,172.753422,182.792571,182.115725,0.676668,0.007664,1.812038e+06,2.777844e+06,9.627107,48.639364,0.001755,0.005908,0.006190
元器件,20.081742,21.226703,19.134599,20.033426,19.998921,0.034506,0.000367,1.199445e+05,1.507868e+05,-0.765584,32.703537,0.001755,-0.001388,-0.001564
全国地产,2062.427046,2140.597400,1995.174155,2064.841752,2065.064835,-0.224212,0.001150,1.672641e+06,4.460160e+06,0.724953,42.353996,0.001755,-0.000605,-0.001260
其他商业,74.713599,78.375824,71.535759,74.910982,74.687662,0.223359,0.004491,3.067152e+05,3.474380e+05,3.352963,47.198449,0.001755,0.002736,0.003093
农业综合,32.459953,34.254851,31.048125,32.294073,32.338493,-0.044409,-0.000474,6.087670e+05,5.332754e+05,1.501678,36.472447,0.001949,-0.002441,-0.002546
区域地产,120.776327,126.410055,116.048496,120.579937,120.419638,0.160254,0.000455,6.130420e+05,5.614498e+05,-5.801097,39.138012,0.001792,-0.001336,-0.001406
医药商业,183.780553,190.704991,177.278093,183.260333,183.341161,-0.080750,-0.001272,1.250399e+05,5.887449e+05,1.544864,42.146085,0.001755,-0.003027,-0.002778
家用电器,119.738616,127.036506,114.085255,119.733659,119.781459,-0.047827,0.003018,2.039494e+06,1.510662e+06,3.902710,39.239790,0.001755,0.001263,0.001219
建筑工程,41.884722,43.978336,40.138513,41.880805,41.735175,0.145608,-0.000147,3.129257e+05,1.282355e+05,-5.108515,39.942271,0.001755,-0.001902,-0.001114


# 标准化

将中性化处理后的因子暴露度序列减去其现在的均值、除以其标准差，得到一个新的近似服从N(0,1)分布的序列。
所以要对一个列中的所有的数据做，本来考虑用行业，但是后来思考后，觉得整体服从正态分布更重要。

In [51]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_train[feature_names])
df_train[feature_names] = scaler.transform(df_train[feature_names])
df_test[feature_names] = scaler.transform(df_test[feature_names])
df_train

/Users/piginzoo/software/py3/lib/python3.8/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,ts_code,name,area,industry,market,list_status,list_date,delist_date,period,trade_date,...,pre_close,change,pct_chg,volume,amount,MACD,KDJ,pct_chg_hs300,rm_rf,target
0,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180105,...,1408.5942,5.3160,0.000000,8108913.30,1.092060e+07,NaN,NaN,0.0268,-0.026800,-0.002003
1,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180112,...,1424.5406,15.9450,0.018797,8704112.30,1.152740e+07,NaN,NaN,0.0208,-0.002003,0.077951
2,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180119,...,1564.8685,8.5100,0.092251,12942411.10,1.865851e+07,NaN,NaN,0.0143,0.077951,-0.073076
3,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180126,...,1509.5878,-15.9450,-0.050676,11456924.00,1.660992e+07,NaN,NaN,0.0224,-0.073076,0.025100
4,000001.SZ,平安银行,深圳,银行,主板,L,1991-04-03,None,11425 days,20180202,...,1491.5153,2.1270,0.000000,8115143.20,1.129147e+07,NaN,NaN,-0.0251,0.025100,-0.067172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4031,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210219,...,154.6888,5.4700,0.035631,75696.56,2.970532e+05,-1.162537,-0.375318,-0.0050,0.040631,0.045585
4032,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210226,...,156.2461,-1.0380,-0.030915,171648.18,6.774649e+05,-1.219230,-0.144869,-0.0765,0.045585,0.056864
4033,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210305,...,160.1992,1.6770,0.042964,134960.12,5.399992e+05,-1.174820,0.171089,-0.0139,0.056864,-0.002073
4034,000028.SZ,国药一致,深圳,医药商业,主板,L,1993-08-09,None,10566 days,20210312,...,157.6836,0.2795,-0.024173,111698.55,4.448124e+05,-1.169902,0.188658,-0.0221,-0.002073,0.060972


# 训练

In [58]:
from sklearn import linear_model

df_train = df_train[feature_names+['target']]
df_train.dropna(inplace=True)
df_test = df_train[feature_names+['target']]
df_test.dropna(inplace=True)

X_train = df_train[feature_names].values
X_test = df_test[feature_names].values
y_train = df_train.target
y_test = df_test.target

reg = linear_model.LinearRegression()
print(X_train.shape,y_train.shape)
model = reg.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred

(3216, 2) (3216,)


array([0.00365194, 0.00418338, 0.0039578 , ..., 0.00135401, 0.00136457,
       0.00199884])